# Introducing MLib package of PySpark 

## 第5章 MLlib介绍

## Load and transform the data

Just like in the previous chapter, we first specify the schema of our dataset.

```bash
wget http://www.tomdrabas.com/data/LearningPySpark/births_train.csv.gz;
cd /root/jupyternotebook/learningPySpark-master/Chapter05/;
hadoop fs -put ./births_train.csv.gz /jupyter-notebook-data/;
```
MLlib代表机器学习库。即使MLlib现在处于维护模式，即它没有被积极开发（并且很可能在以后被弃用），但我们保证至少会覆盖该库的一些特性。此外，MLlib是目前唯一支持流媒体训练模型的库。

从Spark 2.0开始，ML是主要的机器学习库，它对DataFrame进行操作，而不像MLlib那样对RDD进行操作。

MLlib概括了其公开三个核心机器学习功能：
- 数据准备：特征提取、变换、选择、分类特征的散列和一些自然语言处理方法。
- 机器学习算法：实现了一些流行和高级的回归，分类和聚类算法。
- 实用程序：统计方法，如描述性统计、卡方检验、线性代数（稀疏稠密矩阵和向量）和模型评估方法。

如您所见，可用功能面板可以让您执行几乎所有的基础数据科学任务。

在本章中，我们将构建两个分类模型：线性回归和随机森林。我们将使用从http://www.cdc.gov/nchs/data_access/vitalstatsonline.htm 下载的美国2014年和2015年出生数据的一部分。我们从300个变量中选择了85个特征，我们将使用它们来构建我们的模型。此外，在总数近799万条记录中，我们选择了45429条记录的平衡样本：22080条婴儿死亡的记录，23349条婴儿活着的记录。

In [1]:
import pyspark.sql.types as typ

labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.StringType()),
    ('BIRTH_YEAR', typ.IntegerType()),
    ('BIRTH_MONTH', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('MOTHER_RACE_6CODE', typ.StringType()),
    ('MOTHER_EDUCATION', typ.StringType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('FATHER_EDUCATION', typ.StringType()),
    ('MONTH_PRECARE_RECODE', typ.StringType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_BMI_RECODE', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.StringType()),
    ('DIABETES_GEST', typ.StringType()),
    ('HYP_TENS_PRE', typ.StringType()),
    ('HYP_TENS_GEST', typ.StringType()),
    ('PREV_BIRTH_PRETERM', typ.StringType()),
    ('NO_RISK', typ.StringType()),
    ('NO_INFECTIONS_REPORTED', typ.StringType()),
    ('LABOR_IND', typ.StringType()),
    ('LABOR_AUGM', typ.StringType()),
    ('STEROIDS', typ.StringType()),
    ('ANTIBIOTICS', typ.StringType()),
    ('ANESTHESIA', typ.StringType()),
    ('DELIV_METHOD_RECODE_COMB', typ.StringType()),
    ('ATTENDANT_BIRTH', typ.StringType()),
    ('APGAR_5', typ.IntegerType()),
    ('APGAR_5_RECODE', typ.StringType()),
    ('APGAR_10', typ.IntegerType()),
    ('APGAR_10_RECODE', typ.StringType()),
    ('INFANT_SEX', typ.StringType()),
    ('OBSTETRIC_GESTATION_WEEKS', typ.IntegerType()),
    ('INFANT_WEIGHT_GRAMS', typ.IntegerType()),
    ('INFANT_ASSIST_VENTI', typ.StringType()),
    ('INFANT_ASSIST_VENTI_6HRS', typ.StringType()),
    ('INFANT_NICU_ADMISSION', typ.StringType()),
    ('INFANT_SURFACANT', typ.StringType()),
    ('INFANT_ANTIBIOTICS', typ.StringType()),
    ('INFANT_SEIZURES', typ.StringType()),
    ('INFANT_NO_ABNORMALITIES', typ.StringType()),
    ('INFANT_ANCEPHALY', typ.StringType()),
    ('INFANT_MENINGOMYELOCELE', typ.StringType()),
    ('INFANT_LIMB_REDUCTION', typ.StringType()),
    ('INFANT_DOWN_SYNDROME', typ.StringType()),
    ('INFANT_SUSPECTED_CHROMOSOMAL_DISORDER', typ.StringType()),
    ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', typ.StringType()),
    ('INFANT_BREASTFED', typ.StringType())
]

schema = typ.StructType([
        typ.StructField(e[0], e[1], False) for e in labels
    ])

Next, we load the data.

虽然MLlib是着重为RDD和DStream设计的，但是为了方便转换数据，我们将读取数据并将其转换为DataFrame。

接下来，我们来加载数据。.read.csv（...）方法可以读取未压缩的或（如同本例中）“GZipped”压缩的逗号分隔的值。参数header设置为true，代表第一行包含头，而且我们使用schema来指定正确的数据类型：

In [2]:
try:
    sc.stop()
except:
    pass
#---------------------------#
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder \
    .master('spark://master:7077') \
    .appName('Introducing MLib package of PySpark') \
    .getOrCreate()

In [3]:
births = spark.read.csv('/jupyter-notebook-data/births_train.csv.gz', 
                        header=True, 
                        schema=schema)

Specify our recode dictionary.

首先定义重编码字典：

In [4]:
recode_dictionary = {
    'YNU': {
        'Y': 1,
        'N': 0,
        'U': 0
    }
}

Our goal is to predict whether the `'INFANT_ALIVE_AT_REPORT'` is either 1 or 0. Thus, we will drop all of the features that relate to the infant.

我们本章的目标是预测“INFANT_ALIVE_AT_REPORT”是1还是0。因此，我们将丢弃与婴儿相关的所有特征，而仅仅基于与其母亲、父亲和出生地点相关的特征来预测婴儿的存活机会：

In [5]:
selected_features = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]

births_trimmed = births.select(selected_features)

Specify the recoding methods.

在我们的数据集中有大量的特征，它们的值是Yes/No/Unknown；我们将仅仅把Yes编码为1，其他值设置为0。

还有个小问题是，母亲吸烟的数量如何编码：因为0意味着母亲在怀孕前或怀孕期间没有吸烟；1～97之间代表的是母亲实际吸烟数量；98代表母亲实际吸烟数量是98或更多；而99代表母亲实际吸烟数量未知，我们将假设未知状态为0，并以此重新编码。

接下来我们将指定我们的重新编码方法：

recode 方法从 recode_dictionary（给出键）查找正确的键，并返回更正的值。

correct_cig 方法检查如下，当 feat 特征值不等于 99 时，返回特征值；如果值等于 99，我们则得到 0。

我们不能直接在 DataFrame 上使用 recode 函数，它需要转换为 Spark 可理解的 UDF（用户定义函数）。rec_integer 函数功能如下：通过传递我们指定的 recode **函数**及指定**返回值**数据类型，我们可以用它来重新编码 Yes/No/Unknown 特征。

In [6]:
import pyspark.sql.functions as func

#recode_dictionary = {
#    'YNU': {
#        'Y': 1,
#        'N': 0,
#        'U': 0
#    }
#}

def recode(col, key):        
    return recode_dictionary[key][col] 

def correct_cig(feat):
    return func \
        .when(func.col(feat) != 99, func.col(feat))\
        .otherwise(0)
rec_integer = func.udf(recode, typ.IntegerType())

Correct the features related to the number of smoked cigarettes.

.withColumn（...）方法用列名作为其第一个参数，用转换作为第二个参数。在以前的例子中，我们不会创建新列，而是重用相同的列。

In [7]:
births_transformed = births_trimmed \
    .withColumn('CIG_BEFORE', correct_cig('CIG_BEFORE'))\
    .withColumn('CIG_1_TRI', correct_cig('CIG_1_TRI'))\
    .withColumn('CIG_2_TRI', correct_cig('CIG_2_TRI'))\
    .withColumn('CIG_3_TRI', correct_cig('CIG_3_TRI'))

Figure out which Yes/No/Unknown features are.

现在我们将集中更正 Yes/No/Unknown 特征。首先，用下面的代码段来确定有哪些：

首先，我们创建了一个包含列名称和相应数据类型的元组（cols）列表。接下来，我们循环遍历这些列表，并计算所有字符串列的不同值；如果“Y”在返回的列表中，我们将列名追加到YNU_cols列表。

In [8]:
cols = [(col.name, col.dataType) for col in births_trimmed.schema]

YNU_cols = []

for i, s in enumerate(cols):
    if s[1] == typ.StringType():
        dis = births.select(s[0]) \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

        if 'Y' in dis:
            YNU_cols.append(s[0])
            print (s[0])
# 其实这里说的是把每一列取值弄成一个set，看看Y在不在其中

INFANT_ALIVE_AT_REPORT
DIABETES_PRE
DIABETES_GEST
HYP_TENS_PRE
HYP_TENS_GEST
PREV_BIRTH_PRETERM


In [50]:
cols = [(col.name, col.dataType) for col in births_trimmed.schema]
for i, s in enumerate(cols):
    print (i)
    print (s)
    print (type(i))
    print (type(s))
    break

0
('INFANT_ALIVE_AT_REPORT', StringType)
<class 'int'>
<class 'tuple'>


In [51]:
dis_test = births.select('INFANT_ALIVE_AT_REPORT').distinct()\
.rdd.map(lambda row: row[0]).collect()

print (dis_test)
print (type(dis_test))

['Y', 'N']
<class 'list'>


In [40]:
print (births_trimmed,'\n')
print (type(births_trimmed))
births_trimmed.show(1)

DataFrame[INFANT_ALIVE_AT_REPORT: string, BIRTH_PLACE: string, MOTHER_AGE_YEARS: int, FATHER_COMBINED_AGE: int, CIG_BEFORE: int, CIG_1_TRI: int, CIG_2_TRI: int, CIG_3_TRI: int, MOTHER_HEIGHT_IN: int, MOTHER_PRE_WEIGHT: int, MOTHER_DELIVERY_WEIGHT: int, MOTHER_WEIGHT_GAIN: int, DIABETES_PRE: string, DIABETES_GEST: string, HYP_TENS_PRE: string, HYP_TENS_GEST: string, PREV_BIRTH_PRETERM: string] 

<class 'pyspark.sql.dataframe.DataFrame'>
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_BEFORE|CIG_1_TRI|CIG_2_TRI|CIG_3_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|MOTHER_DELIVERY_WEIGHT|MOTHER_WEIGHT_GAIN|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+------

DataFrames can transform the features *in bulk* while selecting features.

我们选择“INFANT_NICU_ADMISSION”列，并且将该特征的名称传递给rec_integer方法。我们还将新转换的列的别名称为“INFANT_NICU_ADMISSION_RECODE”。这样我们还可确保UDF按预期工作。

#### 上文的代码

```py
recode_dictionary = {
    'YNU': {
        'Y': 1,
        'N': 0,
        'U': 0
    }
}

def recode(col, key):        
    return recode_dictionary[key][col] 

def correct_cig(feat):
    return func \
        .when(func.col(feat) != 99, func.col(feat))\
        .otherwise(0)
rec_integer = func.udf(recode, typ.IntegerType())
```
**为什么**rec_integer能够这样使用？？

推测：注册为udf后，所有参数都是整列！这里的调用方法是
```python3
rec_integer('INFANT_NICU_ADMISSION', func.lit('YNU'))
```
说明第一个参数是选定的一整列，第二个参数是所有cell的值都为`YNU`的一列！然后函数读取YNU对应的0或1，并根据第一列相应cell的value进行判断并返回值。

In [9]:
births.select([
        'INFANT_NICU_ADMISSION', 
        rec_integer(
            'INFANT_NICU_ADMISSION', func.lit('YNU')
        ).alias('INFANT_NICU_ADMISSION_RECODE') ]
     ).take(5)
# 增加列有2种方法，一种是基于现在的列计算；
#一种是用pyspark.sql.functions的lit()增加常数列。

[Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION='N', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0)]

In [53]:
print (func.lit('YNU'))
print (type(func.lit('YNU')))

Column<b'YNU'>
<class 'pyspark.sql.column.Column'>


Transform all the `YNU_cols` in one using a list of transformations.

In [10]:
exprs_YNU = [
    rec_integer(x, func.lit('YNU')).alias(x) 
    if x in YNU_cols 
    else x 
    for x in births_transformed.columns
]

births_transformed = births_transformed.select(exprs_YNU)

In [48]:
for i in exprs_YNU:
    print (i,'\n',type(i),'\n')
print ('\n',type(exprs_YNU))

Column<b'recode(INFANT_ALIVE_AT_REPORT, YNU) AS `INFANT_ALIVE_AT_REPORT`'> 
 <class 'pyspark.sql.column.Column'> 

BIRTH_PLACE 
 <class 'str'> 

MOTHER_AGE_YEARS 
 <class 'str'> 

FATHER_COMBINED_AGE 
 <class 'str'> 

CIG_BEFORE 
 <class 'str'> 

CIG_1_TRI 
 <class 'str'> 

CIG_2_TRI 
 <class 'str'> 

CIG_3_TRI 
 <class 'str'> 

MOTHER_HEIGHT_IN 
 <class 'str'> 

MOTHER_PRE_WEIGHT 
 <class 'str'> 

MOTHER_DELIVERY_WEIGHT 
 <class 'str'> 

MOTHER_WEIGHT_GAIN 
 <class 'str'> 

Column<b'recode(DIABETES_PRE, YNU) AS `DIABETES_PRE`'> 
 <class 'pyspark.sql.column.Column'> 

Column<b'recode(DIABETES_GEST, YNU) AS `DIABETES_GEST`'> 
 <class 'pyspark.sql.column.Column'> 

Column<b'recode(HYP_TENS_PRE, YNU) AS `HYP_TENS_PRE`'> 
 <class 'pyspark.sql.column.Column'> 

Column<b'recode(HYP_TENS_GEST, YNU) AS `HYP_TENS_GEST`'> 
 <class 'pyspark.sql.column.Column'> 

Column<b'recode(PREV_BIRTH_PRETERM, YNU) AS `PREV_BIRTH_PRETERM`'> 
 <class 'pyspark.sql.column.Column'> 


 <class 'list'>


Let's check if we got it correctly.

In [11]:
births_transformed.select(YNU_cols[-5:]).show(5)

+------------+-------------+------------+-------------+------------------+
|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+------------+-------------+------------+-------------+------------------+
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 1|
|           0|            0|           0|            0|                 0|
+------------+-------------+------------+-------------+------------------+
only showing top 5 rows



## Get to know your data

### Descriptive statistics

We will use the `colStats(...)` method.

为了清晰地建造一个统计模型，需要深入了解数据集的知识。不了解数据便来建造一个成功的模型是有可能的，但是接下来的任务会更艰巨，其需要更多的技术资源来测试所有可能的特征组合。因此在花了80%的时间清理数据后，我们接下来会用15%的时间来了解数据。

我通常从描述性统计开始。虽然DataFrame公开了.describe（）方法，但由于我们正在使用MLlib，所以我们将采取.colStats（...）方法。

该方法使用RDD的数据来计算MultivariateStatisticalSummary对象的描述性统计信息，并返回MultivariateStatisticalSummary对象，该对象包含如下描述性统计信息：
- count（）：行数
- max（）：列中的最大值
- mean（）：列的所有值的平均值
- min（）：列中的最小值
- normL1（）：列中值的L1-Norm值
- normL2（）：列中值的L2-Norm值
- numNonzeros（）：列中非零值的数量
- variance（）：列中值的方差值

In [12]:
import pyspark.mllib.stat as st
import numpy as np

numeric_cols = ['MOTHER_AGE_YEARS','FATHER_COMBINED_AGE',
                'CIG_BEFORE','CIG_1_TRI','CIG_2_TRI','CIG_3_TRI',
                'MOTHER_HEIGHT_IN','MOTHER_PRE_WEIGHT',
                'MOTHER_DELIVERY_WEIGHT','MOTHER_WEIGHT_GAIN'
               ]

numeric_rdd = births_transformed\
                       .select(numeric_cols)\
                       .rdd \
                       .map(lambda row: [e for e in row])
# 直接在RDD上调用map，传递的参数是一整行？

mllib_stats = st.Statistics.colStats(numeric_rdd)

for col, m, v in zip(numeric_cols, 
                     mllib_stats.mean(), 
                     mllib_stats.variance()):
    print('{0}: \t{1:.2f} \t {2:.2f}'.format(col, m, np.sqrt(v)))

MOTHER_AGE_YEARS: 	28.30 	 6.08
FATHER_COMBINED_AGE: 	44.55 	 27.55
CIG_BEFORE: 	1.43 	 5.18
CIG_1_TRI: 	0.91 	 3.83
CIG_2_TRI: 	0.70 	 3.31
CIG_3_TRI: 	0.58 	 3.11
MOTHER_HEIGHT_IN: 	65.12 	 6.45
MOTHER_PRE_WEIGHT: 	214.50 	 210.21
MOTHER_DELIVERY_WEIGHT: 	223.63 	 180.01
MOTHER_WEIGHT_GAIN: 	30.74 	 26.23


正如你所看到的，与父亲相比，母亲年龄更小：母亲的平均年龄是28岁，而父亲的平均年龄则超过44岁。一个好的现象（至少对于一些婴儿来说）是许多母亲怀孕后开始戒烟；而令人吃惊的是，有些母亲还是持续吸烟。

For the categorical variables we will calculate the frequencies of their values.

对于分类变量，我们将计算其值的频率：

In [13]:
categorical_cols = [e for e in births_transformed.columns 
                    if e not in numeric_cols]

categorical_rdd = births_transformed\
                       .select(categorical_cols)\
                       .rdd \
                       .map(lambda row: [e for e in row])

# groupBy算子接收一个函数，这个函数返回的值作为key，
# 然后通过这个key来对里面的元素进行分组。
# .groupBy(lambda row: row[i]) 应该是使用第i列的数据进行分组           
for i, col in enumerate(categorical_cols):
    agg = categorical_rdd \
        .groupBy(lambda row: row[i]) \
        .map(lambda row: (row[0], len(row[1])))
# 推测，groupBy产生的数据格式就是(key , count_key)

    print(col, sorted(agg.collect(), 
                      key=lambda el: el[1], 
                      reverse=True))

INFANT_ALIVE_AT_REPORT [(1, 23349), (0, 22080)]
BIRTH_PLACE [('1', 44558), ('4', 327), ('3', 224), ('2', 136), ('7', 91), ('5', 74), ('6', 11), ('9', 8)]
DIABETES_PRE [(0, 44881), (1, 548)]
DIABETES_GEST [(0, 43451), (1, 1978)]
HYP_TENS_PRE [(0, 44348), (1, 1081)]
HYP_TENS_GEST [(0, 43302), (1, 2127)]
PREV_BIRTH_PRETERM [(0, 43088), (1, 2341)]


In [65]:
births_transformed.show(2)
print ('----------------')
print (type(categorical_rdd))
categorical_rdd.take(5)

+----------------------+-----------+----------------+-------------------+---------+----------------+-----------------+------------+-------------+------------+-------------+------------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_1_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+-----------+----------------+-------------------+---------+----------------+-----------------+------------+-------------+------------+-------------+------------------+
|                     0|          1|              29|                 99|        0|              99|              999|           0|            0|           0|            0|                 0|
|                     0|          1|              22|                 29|        0|              65|              180|           0|            0|           0|            0|                 0|
+----------------------+-----------+----

[[0, '1', 0, 0, 0, 0, 0],
 [0, '1', 0, 0, 0, 0, 0],
 [0, '1', 0, 0, 0, 0, 0],
 [0, '1', 0, 0, 0, 0, 1],
 [0, '1', 0, 0, 0, 0, 0]]

In [64]:
for i, col in enumerate(categorical_cols):
    agg = categorical_rdd \
        .groupBy(lambda row: row[i]) \
        .map(lambda row: (row[0], len(row[1])))
    print (i,'\t',col) 
    print (agg)
    break
    

0 	 INFANT_ALIVE_AT_REPORT
PythonRDD[573] at RDD at PythonRDD.scala:53


RDD.map()示意
![](https://upload-images.jianshu.io/upload_images/12486617-47e5a6fc799795c1.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/822/format/webp)

### Correlations

Correlations between our features.

下面的代码将计算相关性矩阵，并且只打印那些相关性系数大于0.5的特征，即corrs>0.5所限制的部分。

In [14]:
corrs = st.Statistics.corr(numeric_rdd)

for i, el in enumerate(corrs > 0.5):
    correlated = [
        (numeric_cols[j], corrs[i][j]) 
        for j, e in enumerate(el) 
        if e == 1.0 and j != i]
    
    if len(correlated) > 0:
        for e in correlated:
            print('{0}-to-{1}: {2:.2f}' \
                  .format(numeric_cols[i], e[0], e[1]))

CIG_BEFORE-to-CIG_1_TRI: 0.83
CIG_BEFORE-to-CIG_2_TRI: 0.72
CIG_BEFORE-to-CIG_3_TRI: 0.62
CIG_1_TRI-to-CIG_BEFORE: 0.83
CIG_1_TRI-to-CIG_2_TRI: 0.87
CIG_1_TRI-to-CIG_3_TRI: 0.76
CIG_2_TRI-to-CIG_BEFORE: 0.72
CIG_2_TRI-to-CIG_1_TRI: 0.87
CIG_2_TRI-to-CIG_3_TRI: 0.89
CIG_3_TRI-to-CIG_BEFORE: 0.62
CIG_3_TRI-to-CIG_1_TRI: 0.76
CIG_3_TRI-to-CIG_2_TRI: 0.89
MOTHER_PRE_WEIGHT-to-MOTHER_DELIVERY_WEIGHT: 0.54
MOTHER_PRE_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.65
MOTHER_DELIVERY_WEIGHT-to-MOTHER_PRE_WEIGHT: 0.54
MOTHER_DELIVERY_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.60
MOTHER_WEIGHT_GAIN-to-MOTHER_PRE_WEIGHT: 0.65
MOTHER_WEIGHT_GAIN-to-MOTHER_DELIVERY_WEIGHT: 0.60


In [69]:
print (corrs)
for x,y in enumerate(corrs > 0.5):
    print (x,'\t',y)

[[ 1.00000000e+00 -3.52028046e-02 -6.41010571e-02 -4.52544240e-02
  -3.35687670e-02 -2.73199103e-02  4.19113600e-02  2.85203229e-02
   2.23328210e-02  1.46244230e-02]
 [-3.52028046e-02  1.00000000e+00  8.79929846e-02  9.43624397e-02
   9.14375554e-02  7.61406630e-02  8.62027558e-02  1.27899779e-01
   9.75061365e-02  3.54820308e-02]
 [-6.41010571e-02  8.79929846e-02  1.00000000e+00  8.25531201e-01
   7.22135074e-01  6.23033578e-01 -1.08714989e-02 -2.64243555e-02
  -4.67241960e-03 -1.18809418e-02]
 [-4.52544240e-02  9.43624397e-02  8.25531201e-01  1.00000000e+00
   8.65457361e-01  7.59919819e-01 -6.38108725e-03 -1.23276110e-02
  -1.12750503e-03 -1.48184443e-02]
 [-3.35687670e-02  9.14375554e-02  7.22135074e-01  8.65457361e-01
   1.00000000e+00  8.93075782e-01 -2.76545239e-03 -6.06184326e-03
   1.46010680e-03 -1.43591588e-02]
 [-2.73199103e-02  7.61406630e-02  6.23033578e-01  7.59919819e-01
   8.93075782e-01  1.00000000e+00 -9.37520764e-04 -3.77646699e-03
   4.83568294e-03 -6.37879421e-03

We can drop most of highly correlated features. 

如你所见，“CIG_...”特征是高度相关的，所以它们中的大部分我们用不到。由于我们要尽快预测婴儿的生存机会，所以我们只会保留“CIG_1_TRI”。另外，如预期的那样，重量特征也是高度相关的，我们只会保留“MOTHER_PRE_WEIGHT”：

In [15]:
features_to_keep = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_1_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]
births_transformed = births_transformed.select([e for e in features_to_keep])

### Statistical testing

Run a Chi-square test to determine if there are significant differences for categorical variables.

我们无法计算分类特征的相关性。然而，我们可以进行卡方检验来确定是否存在显著差异。
以下是如何使用MLlib的.chiSqTest（...）方法：

卡方检验就是统计样本的实际观测值与理论推断值之间的偏离程度，实际观测值与理论推断值之间的偏离程度就决定卡方值的大小，如果卡方值越大，二者偏差程度越大；反之，二者偏差越小；若两个值完全相等时，卡方值就为0，表明理论值完全符合。

我们循环遍历所有的分类变量，并通过“INFANT_ALIVE_AT_REPORT”特征进行转换，以获得计数。接下来，我们将它们转换为RDD，因此我们可以使用pyspark.mllib.linalg模块将它们转换成矩阵。
.Matrices.dense（...）方法的第一个参数指定矩阵的行数，在我们的例子中，这是分类特征的不同值的长度。

第二个参数指定列数：因为我们的“INFANT_ALIVE_AT_REPORT”目标变量只有两个值，所以列数为二。
最后一个参数是要转换为矩阵的值的列表。
这是一个更清楚的例子：

In [70]:
print (ln.Matrices.dense(3,2,[1,2,3,4,5,6]))

DenseMatrix([[1., 4.],
             [2., 5.],
             [3., 6.]])


In [16]:
import pyspark.mllib.linalg as ln

for cat in categorical_cols[1:]:
    agg = births_transformed \
        .groupby('INFANT_ALIVE_AT_REPORT') \
        .pivot(cat) \
        .count()    
    
    agg_rdd = agg \
        .rdd\
        .map(lambda row: (row[1:])) \
        .flatMap(lambda row: 
                 [0 if e == None else e for e in row]) \
        .collect()

    row_length = len(agg.collect()[0]) - 1
    agg_dense = ln.Matrices.dense(row_length, 2, agg_rdd)
    
    test = st.Statistics.chiSqTest(agg_dense)
    print(cat, round(test.pValue, 4))

BIRTH_PLACE 0.0
DIABETES_PRE 0.0
DIABETES_GEST 0.0
HYP_TENS_PRE 0.0
HYP_TENS_GEST 0.0
PREV_BIRTH_PRETERM 0.0


我们的测试表明，所有的特征应该是显著不同的，应该有助于我们预测婴儿的生存机会

In [79]:
# 测试两个
for cat in categorical_cols[1:3]:
    agg = births_transformed \
        .groupby('INFANT_ALIVE_AT_REPORT') \
        .pivot(cat) \
        .count()    
# pivot 行转列，结合groupby语句，就是针对cat中的每一个不同的值，新建列，列名为值，
# 列中的value是count

    print (cat)
    print(type(agg))
    agg.show()
    
    # agg_rdd是list，把agg中的第一列以外的数据抽出来，flatmap为list
    agg_rdd = agg \
        .rdd\
        .map(lambda row: (row[1:])) \
        .flatMap(lambda row: 
                 [0 if e == None else e for e in row]) \
        .collect()

    print(type(agg_rdd))
    print (agg_rdd)
    
    row_length = len(agg.collect()[0]) - 1
    agg_dense = ln.Matrices.dense(row_length, 2, agg_rdd)
    # dense 就是把下面的两行的数据进行转置操作，变成两列的数据。进行卡方检验。
    print(type(agg_dense))

BIRTH_PLACE
<class 'pyspark.sql.dataframe.DataFrame'>
+----------------------+-----+---+---+---+---+---+---+----+
|INFANT_ALIVE_AT_REPORT|    1|  2|  3|  4|  5|  6|  7|   9|
+----------------------+-----+---+---+---+---+---+---+----+
|                     1|22995|113|158| 39| 19|  2| 23|null|
|                     0|21563| 23| 66|288| 55|  9| 68|   8|
+----------------------+-----+---+---+---+---+---+---+----+

<class 'list'>
[22995, 113, 158, 39, 19, 2, 23, 0, 21563, 23, 66, 288, 55, 9, 68, 8]
<class 'pyspark.mllib.linalg.DenseMatrix'>
DIABETES_PRE
<class 'pyspark.sql.dataframe.DataFrame'>
+----------------------+-----+---+
|INFANT_ALIVE_AT_REPORT|    0|  1|
+----------------------+-----+---+
|                     1|23178|171|
|                     0|21703|377|
+----------------------+-----+---+

<class 'list'>
[23178, 171, 21703, 377]
<class 'pyspark.mllib.linalg.DenseMatrix'>


## Create the final dataset

### Create an RDD of `LabeledPoint`s

We will use a hashing trick to encode the `'BIRTH_PLACE'` feature.

因此，现在可以创建用于构建我们的模型的最终数据集了。我们要把DataFrame转换为LabeledPoint的RDD。

LabeledPoint是一种MLlib的数据结构，用于训练机器学习模型。它由两个属性组成：标签和特征。
标签是我们的目标变量，特征可以是NumPy数组、列表、pyspark.mllib.linalg.SparseVector、pyspark.mllib.linalg.DenseVector或scipy.sparse列矩阵。

在构建最终数据集之前，首先需要解决一个最后的障碍：“BIRTH_PLACE”特征仍然是一个字符串。其他分类变量都可以直接使用（因为它们现在是虚拟变量），故我们将使用散列技巧来编码“BIRTH_PLACE”特征：

首先，我们创建哈希模型。因为我们的特征有七个级别，所以我们使用与哈希处理中相同多的特征。接下来，真正要做的是使用该模型将“BIRTH_PLACE”特征转换为SparseVector。如果您的数据集有许多列，但是在一行中只有少数数据具有非零值，则这种数据结构是首选的。然后，我们将所有特征结合在一起，最终创建一个LabeledPoint。

In [17]:
import pyspark.mllib.feature as ft
import pyspark.mllib.regression as reg

hashing = ft.HashingTF(7)
# 为什么是7？BIRTH_PLACE 有8个不同的值

births_hashed = births_transformed \
    .rdd \
    .map(lambda row: [ # row 是行
            list(hashing.transform(row[1]).toArray()) # 因为 BIRTH_PLACE 是第一列
                if col == 'BIRTH_PLACE' 
                else row[i] 
            for i, col 
            in enumerate(features_to_keep)]) \
    .map(lambda row: [[e] if type(e) == int else e 
                      for e in row]) \
    .map(lambda row: [item for sublist in row   # ？？？
                      for item in sublist]) \
    .map(lambda row: reg.LabeledPoint(
            row[0], 
            ln.Vectors.dense(row[1:]))
        )

### Split into training and testing

Before we move to the modeling stage, we need to split our dataset into two sets: one we'll use for training and another one for testing.

In [18]:
births_train, births_test = births_hashed.randomSplit([0.6, 0.4])

In [92]:
print (type(births_train))
for i in births_train.collect()[:5]:
    print (i)

<class 'pyspark.rdd.PipelinedRDD'>
(0.0,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,99.0,0.0,99.0,999.0,0.0,0.0,0.0,0.0,0.0])
(0.0,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,29.0,0.0,65.0,180.0,0.0,0.0,0.0,0.0,0.0])
(0.0,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,40.0,0.0,63.0,155.0,0.0,0.0,0.0,0.0,0.0])
(0.0,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,99.0,4.0,61.0,110.0,0.0,0.0,0.0,0.0,0.0])
(0.0,[1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,26.0,0.0,64.0,136.0,0.0,0.0,0.0,0.0,0.0])


## Predicting infant survival

### Logistic regression in Spark

MLLib used to provide a logistic regression model estimated using a stochastic gradient descent (SGD) algorithm. This model has been deprecated in Spark 2.0 in favor of the `LogisticRegressionWithLBFGS` model. 

我们终于可以开始预测婴儿的生存机会了。在本节中，我们将构建两个模型：线性分类器（linear classifier）——逻辑回归，和非线性分类器（non-linear classifier）——随机森林。对于前者，我们使用所有特征来做处理，而对于后者，我们使用ChiSqSelector（...）方法选出前四个特征。

逻辑回归从某种程度上说，是构建任何分类模型的基准。MLlib过去使用随机梯度下降（SGD）算法来提供逻辑回归模型的评估。这个模型已经在Spark 2.0中被弃用，而使用LogisticRegressionWithLBFGS模型。

LogisticRegressionWithLBFGS模型使用Limited-memoryBroyden-Fletcher-Goldfarb-Shanno（BFGS）优化算法。这是一种接近于BFGS算法的拟牛顿方法。

训练模型非常简单：我们只需调用.train（...）方法。需要的参数是带有LabeledPoint的RDD；我们还指定了迭代次数，因此运行时间不会太长。

In [19]:
from pyspark.mllib.classification \
    import LogisticRegressionWithLBFGS

LR_Model = LogisticRegressionWithLBFGS \
    .train(births_train, iterations=10)

Let's now use the model to predict the classes for our testing set.

下面的代码段创建了一个RDD，其中每个元素都是一个元组，第一个元素是实际的标签，第二个元素是模型的预测。

In [20]:
LR_results = (
        births_test.map(lambda row: row.label) \
        .zip(LR_Model.predict(births_test.map(lambda row: row.features)))
    ).map(lambda row: (row[0], row[1] * 1.0))

Let's check how well or how bad our model performed.

In [21]:
import pyspark.mllib.evaluation as ev
LR_evaluation = ev.BinaryClassificationMetrics(LR_results)

print('Area under PR: {0:.2f}' \
      .format(LR_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(LR_evaluation.areaUnderROC))
LR_evaluation.unpersist()

Area under PR: 0.80
Area under ROC: 0.63


模型表现相当不错！Precision-Recall曲线下85%的面积指示契合得很好。在这种情况下，我们可能会预测死亡人数略有增加（真正和假正）。而这种情况实际上是一件好事，因为这样可以让医生对怀孕的母亲和婴儿做特别的关注。

受试者工作特性（Receiver-Operating Characteristic）（ROC）曲线下的面积可以理解为：与随机选择的负实例相比，模型排名几率高于随机选择的正实例。63%这个值是可以接受的。

### Selecting only the most predictable features

MLLib allows us to select the most predictable features using a Chi-Square selector.

任何使用较少特征来准确预测类的模型，应始终优于使用复杂特征的模型。MLlib允许使用Chi-Square选择器来选择最可预测的特征。

我们要求选择器从数据集返回四个最具预测性的特征，并使用births_train数据集训练选择器。然后，我们使用该模型从训练和测试的数据集中仅仅提取这些特征。

.ChiSqSelector（...）方法只能用于数字特征；在使用选择器之前，分类变量需要进行散列或伪编码。

In [22]:
selector = ft.ChiSqSelector(4).fit(births_train)

topFeatures_train = (
        births_train.map(lambda row: row.label) \
        .zip(selector.transform(births_train.map(lambda row: row.features)))
    ).map(lambda row: reg.LabeledPoint(row[0], row[1]))

topFeatures_test = (
        births_test.map(lambda row: row.label) \
        .zip(selector.transform(births_test.map(lambda row: row.features)))
    ).map(lambda row: reg.LabeledPoint(row[0], row[1]))

In [95]:
print (type(topFeatures_test))
for i in topFeatures_test.collect()[:5]:
    print (i)

<class 'pyspark.rdd.PipelinedRDD'>
(0.0,[0.0,42.0,60.0,128.0])
(0.0,[0.0,37.0,66.0,150.0])
(0.0,[0.0,25.0,68.0,155.0])
(0.0,[0.0,66.0,65.0,140.0])
(0.0,[0.0,41.0,59.0,106.0])


### Random Forest in Spark

We are now ready to build the random forest model. 

.trainClassifier（...）方法的第一个参数指定训练数据集。参数numClasses表示我们的目标变量有多少类。作为第三个参数，您可以传递一个dictionary，其中键是RDD中分类特征的索引，键值表示分类特征具有的级别数。numTrees指定森林中树的数目。下一个参数告诉模型使用数据集中的所有特征，而不是只保留最具描述性的特征，而最后一个参数指定模型随机部分的种子。

In [23]:
from pyspark.mllib.tree import RandomForest

RF_model = RandomForest \
    .trainClassifier(data=topFeatures_train, 
                     numClasses=2, 
                     categoricalFeaturesInfo={}, 
                     numTrees=6,  
                     featureSubsetStrategy='all',
                     seed=666)

Let's see how well our model did.

In [24]:
RF_results = (
        topFeatures_test.map(lambda row: row.label) \
        .zip(RF_model.predict(topFeatures_test.map(lambda row: row.features)))
    )

RF_evaluation = ev.BinaryClassificationMetrics(RF_results)

print('Area under PR: {0:.2f}' \
      .format(RF_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(RF_evaluation.areaUnderROC))
RF_evaluation.unpersist()

Area under PR: 0.76
Area under ROC: 0.62


Let's see how the logistic regression would perform with reduced number of features.

In [71]:
LR_Model_2 = LogisticRegressionWithLBFGS \
    .train(topFeatures_train, iterations=10)

LR_results_2 = (
        topFeatures_test.map(lambda row: row.label) \
        .zip(LR_Model_2 \
             .predict(topFeatures_test \
                      .map(lambda row: row.features)))
    ).map(lambda row: (row[0], row[1] * 1.0))

LR_evaluation_2 = ev.BinaryClassificationMetrics(LR_results_2)

print('Area under PR: {0:.2f}' \
      .format(LR_evaluation_2.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(LR_evaluation_2.areaUnderROC))
LR_evaluation_2.unpersist()

Area under PR: 0.83
Area under ROC: 0.63


在本章中，我们浏览了PySpark的MLlib软件包的功能。虽然该软件包目前处于维护模式，并且没有被积极地推进，但了解如何使用仍然是有益处的。而且它是目前唯一提供用流数据训练模型的软件包。我们使用了MLlib来清理、转换和熟悉婴儿死亡数据集。使用这些知识，我们成功地建立了两个模型，旨在根据其母亲、父亲和出生地点的信息来预测婴儿生存的机会。